# AHLT - MIRI
# Drugs Interaction Classifier

In [1]:
import nltk # NLTK Library
import xml.etree.ElementTree as ET # ElementTree Library
import os
import pandas as pd
import numpy as np
from xlm_parsers_functions import *

## TODO list
# Train the first ML model that will identify the drug names from a sentence
# Build the necessary data structure that will hold the predictors and the response variable
# Train ML models with that data

## Parsing the XML Files

### DrugBank files

In [2]:
# Use xlm_element.tag to get the name of the xlm element
# Use xlm_element.attrib to get all the attributes of the xlm element as a string

# Give the headers name for the final dataset
headers = ['sentence_id', 'sentence_text', 'e1_id', 'e1_name', 'e1_type', 'e2_id', 'e2_name', 'e2_type', 'interaction']

# Parse the DrugBank Files
drugs_dataset = []
parent_directory = '../LaboCase/Train/DrugBank/'
for filename in os.listdir(parent_directory):
    if filename.endswith(".xml"):
        # Parse the file
        tree = ET.parse(parent_directory + filename)
        # Create a list of lists with the interactions of the file
        drugs_dataset = drugs_dataset + listDDIFromXML(tree.getroot())


DrugBank_df = pd.DataFrame(drugs_dataset, columns=headers)

In [3]:
DrugBank_df

,sentence_id,sentence_text,e1_id,e1_name,e1_type,e2_id,e2_name,e2_type,interaction
0,DDI-DrugBank.d481.s0,"Milk, milk products, and calcium-rich foods or...",DDI-DrugBank.d481.s0.e0,calcium,drug,DDI-DrugBank.d481.s0.e1,EMCYT,brand,true
1,DDI-DrugBank.d419.s0,The concurrent administration of allopurinol a...,DDI-DrugBank.d419.s0.e0,allopurinol,drug,DDI-DrugBank.d419.s0.e1,ampicillin,drug,true
2,DDI-DrugBank.d419.s0,The concurrent administration of allopurinol a...,DDI-DrugBank.d419.s0.e0,allopurinol,drug,DDI-DrugBank.d419.s0.e2,ampicillin,drug,false
3,DDI-DrugBank.d419.s0,The concurrent administration of allopurinol a...,DDI-DrugBank.d419.s0.e1,ampicillin,drug,DDI-DrugBank.d419.s0.e2,ampicillin,drug,false
4,DDI-DrugBank.d419.s1,It is not known whether this potentiation of a...,DDI-DrugBank.d419.s1.e0,ampicillin,drug,DDI-DrugBank.d419.s1.e1,allopurinol,drug,true
5,DDI-DrugBank.d419.s2,"In controlled clinical trials of AUGMENTIN XR,...",DDI-DrugBank.d419.s2.e0,AUGMENTIN XR,brand,DDI-DrugBank.d419.s2.e1,allopurinol,drug,false
6,DDI-DrugBank.d419.s2,"In controlled clinical trials of AUGMENTIN XR,...",DDI-DrugBank.d419.s2.e0,AUGMENTIN XR,brand,DDI-DrugBank.d419.s2.e2,AUGMENTIN XR,brand,false
7,DDI-DrugBank.d419.s2,"In controlled clinical trials of AUGMENTIN XR,...",DDI-DrugBank.d419.s2.e1,allopurinol,drug,DDI-DrugBank.d419.s2.e2,AUGMENTIN XR,brand,false
8,DDI-DrugBank.d419.s4,"However, this sample size is too small to allo...",DDI-DrugBank.d419.s4.e0,AUGMENTIN XR,brand,DDI-DrugBank.d419.s4.e1,allopurinol,drug,false
9,DDI-DrugBank.d419.s5,In common with other broad-spectrum antibiotic...,DDI-DrugBank.d419.s5.e0,broad-spectrum antibiotics,group,DDI-DrugBank.d419.s5.e1,AUGMENTIN XR,brand,false


### MedLine files

In [ ]:
drugs_dataset = []
parent_directory = 'LaboCase/Train/MedLine/'
for filename in os.listdir(parent_directory):
    if filename.endswith(".xml"):
        # Parse the file
        tree = ET.parse(parent_directory + filename)
        # Create a list of lists with the interactions of the file
        drugs_dataset = drugs_dataset + drugs_xml_to_list(tree.getroot())

MedLine_df = pd.DataFrame(drugs_dataset, columns=headers)